## 5. 实践案例

### 5.1 异步网络请求

使用aiohttp实现并发网络请求，展示协程在实际应用中的优势：

In [ ]:
import aiohttp
import asyncio
from datetime import datetime

async def fetch_url(session, url):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.text()
            return f'Error: HTTP {response.status}'
    except Exception as e:
        return f'Error: {str(e)}'

async def fetch_urls():
    urls = [
        'http://example.com',
        'http://example.org',
        'http://example.net'
    ]
    
    async with aiohttp.ClientSession() as session:
        start = datetime.now()
        
        # 使用asyncio.gather并发执行所有请求
        tasks = [fetch_url(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        
        end = datetime.now()
        time_taken = (end - start).total_seconds()
        
        print(f'处理 {len(urls)} 个请求用时: {time_taken:.2f} 秒')
        return results

# 注意：运行此代码需要先安装aiohttp:
# pip install aiohttp

# if __name__ == '__main__':
#     asyncio.run(fetch_urls())

### 5.2 异步上下文管理

展示如何使用异步上下文管理器处理资源的获取和释放：

In [ ]:
import asyncio
from contextlib import asynccontextmanager

@asynccontextmanager
async def async_resource(name):
    # 获取资源
    print(f'获取资源: {name}')
    try:
        await asyncio.sleep(1)  # 模拟资源获取
        yield f'资源{name}'
    finally:
        # 释放资源
        print(f'释放资源: {name}')
        await asyncio.sleep(0.5)  # 模拟资源释放

async def process_resources():
    async with async_resource('A') as res1, \
               async_resource('B') as res2:
        print(f'使用资源: {res1}, {res2}')
        await asyncio.sleep(2)  # 模拟使用资源

# if __name__ == '__main__':
#     asyncio.run(process_resources())

## 6. 最佳实践

### 6.1 常见错误处理

1. **阻塞IO操作**
   - 避免在协程中使用同步IO
   - 使用异步库或运行在线程池

2. **资源管理**
   - 正确关闭连接
   - 使用异步上下文管理器

3. **并发控制**
   - 使用Semaphore控制并发量
   - 避免过多任务

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from functools import partial

class AsyncProcessor:
    def __init__(self, max_concurrent=3):
        self.semaphore = asyncio.Semaphore(max_concurrent)
        self.thread_pool = ThreadPoolExecutor(max_workers=3)
    
    async def process_item(self, item):
        # 使用信号量控制并发
        async with self.semaphore:
            try:
                # 模拟异步处理
                await asyncio.sleep(1)
                
                # 如果有CPU密集型任务，放到线程池
                loop = asyncio.get_event_loop()
                result = await loop.run_in_executor(
                    self.thread_pool,
                    partial(self.cpu_bound_task, item)
                )
                return result
            except Exception as e:
                print(f'处理 {item} 时出错: {e}')
                raise
    
    def cpu_bound_task(self, item):
        # 模拟CPU密集型任务
        return item * 2

async def main():
    processor = AsyncProcessor()
    items = range(5)
    
    try:
        tasks = [processor.process_item(item) for item in items]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        print('处理结果:', results)
    except Exception as e:
        print(f'发生错误: {e}')
    finally:
        processor.thread_pool.shutdown()

# if __name__ == '__main__':
#     asyncio.run(main())

## 7. 总结

协程和异步IO是Python中处理高并发任务的强大工具。主要优势包括：

1. **高效性**
   - 单线程处理高并发
   - 低内存开销
   - 快速的上下文切换

2. **简化开发**
   - 直观的async/await语法
   - 避免回调地狱
   - 易于编写和维护

3. **应用场景**
   - Web应用
   - 网络爬虫
   - 微服务
   - 实时应用

# Python协程和异步IO深度教程

本教程将系统地介绍Python中的协程和异步IO编程，包括：
1. 并发编程基础概念
2. IO多路复用
3. 协程的实现原理
4. 生成器和协程
5. async/await语法
6. 实战案例

## 1. 并发编程基础概念

### 1.1 关键概念对比

- **并发 vs 并行**
  - 并发：一个时间段内，多个程序交替执行
  - 并行：同一时刻，多个程序同时执行

- **同步 vs 异步**
  - 同步：调用IO操作时必须等待结果
  - 异步：调用IO操作时立即返回，结果通过回调或其他方式获取

- **阻塞 vs 非阻塞**
  - 阻塞：调用导致线程挂起
  - 非阻塞：调用立即返回

In [ ]:
import time

# 同步阻塞示例
def sync_blocking():
    print('开始执行IO操作...')
    time.sleep(2)  # 模拟IO操作
    print('IO操作完成')

print('同步阻塞示例：')
sync_blocking()

## 2. 协程基础

协程是用户级的轻量级线程，它的特点是：

1. 可以在函数执行过程中暂停
2. 可以保存当前状态
3. 可以从暂停处继续执行
4. 切换开销远小于线程

In [ ]:
def simple_coroutine():
    print('-> 开始执行')
    x = yield                      # 第一个暂停点
    print('-> 接收到值:', x)
    y = yield x                    # 第二个暂停点
    print('-> 接收到值:', y)

# 使用生成器协程
def demo_coroutine():
    cor = simple_coroutine()
    next(cor)                      # 启动生成器
    print('发送值 10')
    result = cor.send(10)          # 发送值并获取yield的结果
    print('yield返回值:', result)
    try:
        cor.send(20)              # 发送最后一个值
    except StopIteration:
        print('协程执行完成')

demo_coroutine()

## 3. yield from

yield from是Python 3.3引入的语法，提供了子生成器的委派机制：

1. 建立通道：把外部的调用方和内部的子生成器连接起来
2. 自动传递值
3. 自动处理异常
4. 返回值处理

In [ ]:
def sub_gen():
    for i in range(10):
        yield i
    return 'sub_gen完成'

def delegator():
    # yield from会建立双向通道
    result = yield from sub_gen()
    print('子生成器的返回值:', result)

# 运行示例
for item in delegator():
    print('获得值:', item)

## 4. async/await

Python 3.5引入的新语法，使协程编程更加直观：

1. async def 定义原生协程
2. await 等待协程完成
3. 避免了基于生成器的协程复杂性

In [ ]:
import asyncio

async def async_task(name, delay):
    print(f'{name} 开始执行')
    await asyncio.sleep(delay)  # 模拟异步IO
    print(f'{name} 执行完成')
    return f'{name} 的结果'

async def main():
    # 创建多个任务并发执行
    tasks = [
        async_task('任务1', 1),
        async_task('任务2', 2),
        async_task('任务3', 1.5)
    ]
    results = await asyncio.gather(*tasks)
    print('所有任务的结果:', results)

# 运行协程
if __name__ == '__main__':
    asyncio.run(main())